## Rell Autos

Rell autos has been helping it's customers to sell their used cars for more than 5 years now. They have developed their platform on a very large scale. This helps them in attracting more customers who want to sell their cars on rell autos. In order to provide their customers to decide a base price of their used cars, they want a feature to be added that predicts the price of a car based on its details.

### Problem statement: 
Predict the base price of a used car based on details provided.

## Dataset Description

- __name__: Name of the car
- __year__: Manufacturing year of the car
- __selling_price__: Selling price of the car
- __km_driven__: Number of kilometers driven 
- __fuel__: Type of fuel
- __sell_type__: Type of the seller
- __transmission__: Transmission of the car, auto or manual
- __owner__: First owner, Second owner and so on
- __mileage__: Mileage per litre of fuel
- __engine__: Volume of engine in CC
- __max_power__: Power in bhp
- __torque__: Torque power in Nm at certain rpm value
- __seats__: Number of seat__

## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from pandas_profiling import ProfileReport

## Reading Data

In [2]:
# Sepcifiy your bucket, prefix(folder name) and, generate S3 Path
s3_bucket = 'aiml-machine-learning'
s3_prefix = 'Car Prices' #prefix used for data stored within the bucket
s3_path = 's3://{}/{}/'.format(s3_bucket, s3_prefix)

In [3]:
# Read the dataset 
data_key = 'CarDetails.csv'
data_location = '{}{}'.format(s3_path, data_key)
data = pd.read_csv(data_location) 

In [4]:
# Display first five rows
data.head()

name  year  selling_price  km_driven    fuel  \
0        Maruti Swift Dzire VDI  2014         450000     145500  Diesel   
1  Skoda Rapid 1.5 TDI Ambition  2014         370000     120000  Diesel   
2      Honda City 2017-2020 EXi  2006         158000     140000  Petrol   
3     Hyundai i20 Sportz Diesel  2010         225000     127000  Diesel   
4        Maruti Swift VXI BSIII  2007         130000     120000  Petrol   

  seller_type transmission         owner     mileage   engine   max_power  \
0  Individual       Manual   First Owner   23.4 kmpl  1248 CC      74 bhp   
1  Individual       Manual  Second Owner  21.14 kmpl  1498 CC  103.52 bhp   
2  Individual       Manual   Third Owner   17.7 kmpl  1497 CC      78 bhp   
3  Individual       Manual   First Owner   23.0 kmpl  1396 CC      90 bhp   
4  Individual       Manual   First Owner   16.1 kmpl  1298 CC    88.2 bhp   

                     torque  seats  
0            190Nm@ 2000rpm    5.0  
1       250Nm@ 1500-2500rpm    5.0  
2     12.7@ 2,700(kgm@ rpm)    5.0  
3  22.4 kgm at 1750-2750rpm    5.0  
4     11.5@ 4,500(kgm@ rpm)    5.0

### Generating a Report of Data

In [5]:
report=ProfileReport(data)
report.to_notebook_iframe()

In [6]:
report.to_file('car_details.html')

- Which columns are highly **skewed**?
- Which columns are highly **Kurtosis** driven?
- Which columns have Wrong data type?
- What columns seems to have **outliers** based on **min**, **max** and **percentile values**, **IQR range** along with the **standard deviation** and **mean absolute deviation**?
- What columns have missing values? (Check the **Missing Values** section in **Pandas Profiling**)
- What columns have high amount of zero/ infinite percentage and make sure that these zeroes/ infinite are supposed to be there

**For Example:** Weight cannot be zero/ infinite and any percentage of zero/ infinite in column zero is erroneous
- What columns have **high variance** and **standard deviation**?
- Comment on the distribution of the continuous values **(Real Number: ℝ≥0)**
- Do you see any alarming trends in the extreme values (minimum 5 and maximum 5)?
- How many booleans columns are there in the data set and out of those how many are imbalanced?
- Check for **duplicate records** across all columns (**Check Warning Section**)
- How many columns are categorical?
  - Are those categories in sync with the domain categories?
  - Check if all the categories are unique and they represent distinct information
  - Is there any imbalance in the categorical columns?

Based on the above questions and your observations, chart out a plan for **Data Pre-processing** and feature engineering

**Note:** Feature Engineering (Feature Selection and Feature Creation)

- From the **Interaction Tab**, write at least 3 observations that may be very crucial for prediction.


## Top 10 cars for sale

In [7]:
# Create a plotly plot which shows top 10 cars for sale
px.pie(data_frame = data ,names = data.name.value_counts().head(10).index,values = data.name.value_counts().values[0:10],
      title = "Top 10 cars for sale",hole = 0.8
      )

#### Observation:
- Maruti Swift Dzire VDI top selling car
- Most of people own maruti cars

## Year Make of Cars

In [8]:
# Create a plotly plot which shows the frequency manufactured year of cars
plt_data = [
    go.Bar(
        y=data['year'].value_counts().keys(),
        x=data['year'].value_counts(),
        orientation='h',
        text="d",
    )]
layout = go.Layout(
    height=500,
    title='Year Make of Cars',
    hovermode='closest',
    xaxis=dict(title='Votes', ticklen=5, zeroline=False, gridwidth=2, domain=[0.1, 1]),
    yaxis=dict(title='', ticklen=5, gridwidth=2),
    showlegend=False
)
fig = go.Figure(data=plt_data, layout=layout)
fig.show()

#### Observation:
- With growing industries people have more motor vehicles
- Most of the people are reselling their old cars for new ones in recent 5 years

## Fuel Type

In [9]:
# Create a plotly plot for fuel type
px.pie(data_frame = data ,names = data.fuel.value_counts().index,values = data.fuel.value_counts().values,
      title = "Fuel Type " ,hole = 0.7
      )

#### Observation:
- Diesel and petrol cars dominate

## Seller Type

In [10]:
# Function to plot value counts plots
def plot_value_counts_bar(data, col):

    vc = pd.DataFrame(data[col].value_counts())
    vc['cat'] = vc.index
    fig = px.bar(vc, x='cat', y=col, color='cat', title=col)
    fig.update_layout()

    return fig

In [11]:
# Create a plotly value counts plot for seller type 
plot_value_counts_bar(data,'seller_type')

#### Observation:
- There are a few dealer and trustmark dealers on Rell autos but individual sellers dominate by a large count.

## Mileage and Engine

In [12]:
data.head()

name  year  selling_price  km_driven    fuel  \
0        Maruti Swift Dzire VDI  2014         450000     145500  Diesel   
1  Skoda Rapid 1.5 TDI Ambition  2014         370000     120000  Diesel   
2      Honda City 2017-2020 EXi  2006         158000     140000  Petrol   
3     Hyundai i20 Sportz Diesel  2010         225000     127000  Diesel   
4        Maruti Swift VXI BSIII  2007         130000     120000  Petrol   

  seller_type transmission         owner     mileage   engine   max_power  \
0  Individual       Manual   First Owner   23.4 kmpl  1248 CC      74 bhp   
1  Individual       Manual  Second Owner  21.14 kmpl  1498 CC  103.52 bhp   
2  Individual       Manual   Third Owner   17.7 kmpl  1497 CC      78 bhp   
3  Individual       Manual   First Owner   23.0 kmpl  1396 CC      90 bhp   
4  Individual       Manual   First Owner   16.1 kmpl  1298 CC    88.2 bhp   

                     torque  seats  
0            190Nm@ 2000rpm    5.0  
1       250Nm@ 1500-2500rpm    5.0  
2     12.7@ 2,700(kgm@ rpm)    5.0  
3  22.4 kgm at 1750-2750rpm    5.0  
4     11.5@ 4,500(kgm@ rpm)    5.0

In [13]:
# Process the data 
def get_value(data,cols):
    tata={i:[] for i in cols}
    for i in cols:
        for item in data[i]:
            if str(item)=='':
                tata[i].append(np.nan)
            if str(item)!='nan':
                tata[i].append(item.split(' ')[0])
            else:
                tata[i].append(np.nan)
    return tata

# This function will replace the original column with new columns of categories
def update_data(data, cols, dict_data):
    data.drop(cols, inplace=True, axis=1)
    for i in dict_data:
        data[i] = dict_data[i]
    return data

In [14]:
dict_data=get_value(data,['name','mileage','engine','max_power'])
data=update_data(data,['name','mileage','engine','max_power'],dict_data)

In [15]:
data.head()

year  selling_price  km_driven    fuel seller_type transmission  \
0  2014         450000     145500  Diesel  Individual       Manual   
1  2014         370000     120000  Diesel  Individual       Manual   
2  2006         158000     140000  Petrol  Individual       Manual   
3  2010         225000     127000  Diesel  Individual       Manual   
4  2007         130000     120000  Petrol  Individual       Manual   

          owner                    torque  seats     name mileage engine  \
0   First Owner            190Nm@ 2000rpm    5.0   Maruti    23.4   1248   
1  Second Owner       250Nm@ 1500-2500rpm    5.0    Skoda   21.14   1498   
2   Third Owner     12.7@ 2,700(kgm@ rpm)    5.0    Honda    17.7   1497   
3   First Owner  22.4 kgm at 1750-2750rpm    5.0  Hyundai    23.0   1396   
4   First Owner     11.5@ 4,500(kgm@ rpm)    5.0   Maruti    16.1   1298   

  max_power  
0        74  
1    103.52  
2        78  
3        90  
4      88.2

In [16]:
data.engine=data.engine.astype(float)
data.mileage=data.mileage.astype(float)

In [17]:
# Create a plotly plot mileage vs km_driven
me_data=data.groupby('name')[['mileage','engine','km_driven']].agg('mean')
fig=px.scatter(me_data, x="mileage", y="km_driven",size="km_driven", color="mileage",
                 hover_name=me_data.index, log_x=True, size_max=60)
fig.show()

#### Observation:
- Ashok branded vehicles are long lasting

## Cars Brands

In [18]:
plot_value_counts_bar(data,'name')

#### Observation:
- Maruti vehicles dominate